In [1]:
import numpy as np
import numpy.linalg as nl
import scipy
import scipy.linalg as sl
import matplotlib.pyplot as plt
import sys
sys.path.append('./src')

np.set_printoptions(suppress=True)

import warnings
warnings.filterwarnings('ignore')

## Set problem parameters

Solve homogeneous IVP
$$
\frac{\mathrm{d} u(t)}{\mathrm{d} t} = -A u(t),\quad u(0) = \vec{u}_0
$$

In [2]:
nq = 1 #2
dim = 2**nq
rng = np.random.Generator(np.random.PCG64(178983489))
T = 0.1 ## Define the time interval [0,T]

## Define random A and u0
u0 = np.matrix( rng.random((dim,1)) ,dtype=float) ## dtype complex is necessary for scipy to compute complex integral
u0 = u0/np.linalg.norm(u0, ord=2)
A = np.matrix(rng.random((dim,dim)),dtype=float)
A = 0.1*(A-0.5) + 1*np.identity(dim)

A,u0

(matrix([[ 0.98729963, -0.0262057 ],
         [ 0.0278113 ,  1.00247044]]),
 matrix([[0.68870611],
         [0.72504062]]))

In [3]:
## beta and epsilon for LCHS parameters, epislon is the error tolerance
beta = 0.8 # 0< beta < 1
epsilon = 0.1 #0.05

## SciPy solution

In [4]:
### Scipy Homogenous solution
def ode_func_ho(t,u):
        return np.array(-A.dot(u).reshape(-1))[0]

spi_sol_ho = scipy.integrate.solve_ivp(ode_func_ho, [0,T],np.array(u0.reshape(-1))[0], method='RK45')
spi_uT_ho = spi_sol_ho.y[:,-1]
if np.linalg.norm(spi_uT_ho.imag,ord=2) < 1e-12:
    spi_uT_ho = spi_uT_ho.real
spi_uT_ho_norm = spi_uT_ho/np.linalg.norm(spi_uT_ho,ord=2)
spi_uT_ho_norm

array([0.69120677, 0.72265704])

## LCHS in classical computation

In [5]:
from lchs import class_lchs_tihs

exp_op, uT = class_lchs_tihs(A, u0, T, beta, epsilon, trunc_multiplier=1, trotterLH=True, verbose=1)
uT = np.array(uT).reshape(-1)
if np.linalg.norm(uT.imag,ord=2) < 1e-12:
    uT = uT.real
uT_norm = uT/np.linalg.norm(uT,ord=2)

print()
print("  Normalized Homogeneous u(T)=           ", uT_norm)
print("  Normalized SciPy Sol   u(T)=           ", spi_uT_ho_norm)
print("  Normalized Homogeneous solution error u(T)           :", np.linalg.norm(uT_norm - spi_uT_ho_norm,ord=2))

  Preset parameters T =  0.1 beta =  0.8 epsilon =  0.1
  Truncation range [-K,K] K = 3.669573487133956
  Step size h1 = 3.669573487133956
  Number of nodes in [mh_1, (m+1)h_1] Q = 4
  Total number of nodes M = 8
  Truncation error bound = 26.770242137166786
  Quadrature error bound = 0.034698851449614716
  Total error bound = 26.8049409886164
  ||c||_1 = 1.1816978425668454
  ||u0||_2 = 1.0

  Normalized Homogeneous u(T)=            [0.68715488 0.72651096]
  Normalized SciPy Sol   u(T)=            [0.69120677 0.72265704]
  Normalized Homogeneous solution error u(T)           : 0.005592000606936343


## DV LCHS

In [8]:
from lchs import quant_lchs_tihs

uT, lchs_circ_dv, full_state_dv, coeffs_dv, unitaries_dv, coeffs_unrot_dv, unitaries_unrot_dv = quant_lchs_tihs(A, u0, T, beta, epsilon, trunc_multiplier=1, trotterLH=True, verbose=1, rich_return=True)
uT = np.array(uT).reshape(-1)
if np.linalg.norm(uT.imag,ord=2) < 1e-12:
    uT = uT.real
uT_norm = uT/np.linalg.norm(uT,ord=2)

print()
print("  Normalized Quantum LCHS u(T)=          ", uT_norm)
print("  Normalized SciPy Sol   u(T)=           ", spi_uT_ho_norm)
print("  Normalized Quantum LCHS solution error u(T)           :", np.linalg.norm(uT_norm - spi_uT_ho_norm,ord=2) )

  Preset parameters T =  0.1 beta =  0.8 epsilon =  0.1
  Truncation range [-K,K] K = 3.669573487133956
  Step size h1 = 3.669573487133956
  Number of nodes in [mh_1, (m+1)h_1] Q = 4
  Total number of nodes M = 8
  Truncation error bound = 26.770242137166786
  Quadrature error bound = 0.034698851449614716
  Total error bound = 26.8049409886164
  ||c||_1 = 1.1816978425668454
  LCU-Oracle: num_qubits_control= 3 num_qubits_op= 1
  Number of Qubits: 4

  Normalized Quantum LCHS u(T)=           [0.68911474 0.72465224]
  Normalized SciPy Sol   u(T)=            [0.69120677 0.72265704]
  Normalized Quantum LCHS solution error u(T)           : 0.002890914791927233


## CV-DV LCHS with varational qubit-qumode ansatzes for Hamiltonian evolution

In [ ]:
from vbos_lchs import bos_var_lchs_tihs

uT, lchs_circ_cd, circ_op_cd, coeffs_cd, unitaries_cd, coeffs_unrot_cd, unitaries_unrot_cd = bos_var_lchs_tihs(A, u0, T, beta, epsilon, trunc_multiplier=1, trotterLH=True, verbose=1)
uT = np.array(uT).reshape(-1)
if np.linalg.norm(uT.imag,ord=2) < 1e-12:
    uT = uT.real
uT_norm = uT/np.linalg.norm(uT,ord=2)

print()
print("  Normalized Variational Bosonic LCHS u(T)=           ", uT_norm)
print("  Normalized SciPy Sol                u(T)=           ", spi_uT_ho_norm)
print("  Normalized Variational Bosonic LCHS solution error u(T)           :", np.linalg.norm(uT_norm - spi_uT_ho_norm,ord=2) )

  Preset parameters T =  0.1 beta =  0.8 epsilon =  0.1
  Truncation range [-K,K] K = 3.669573487133956
  Step size h1 = 3.669573487133956
  Number of nodes in [mh_1, (m+1)h_1] Q = 4
  Total number of nodes M = 8
  Truncation error bound = 26.770242137166786
  Quadrature error bound = 0.034698851449614716
  Total error bound = 26.8049409886164
  ||c||_1 = 1.1816978425668454
  LCU-Oracle: num_qubits_control= 3 num_qubits_op= 1
  Number of Qubits: 5
>> State preparation for initial condition is DISABLED <<

  Normalized Variational Bosonic LCHS u(T)=            [0.68786645-0.00047234j 0.72583695+0.00049559j]
  Normalized SciPy Sol                u(T)=            [0.69120677 0.72265704]
  Normalized Variational Bosonic LCHS solution error u(T)           : 0.004662431994972372


Looks like there are some errors due to optimization